In [3]:
# ==========================================================
# 🌾 Amrutha - Advanced Agricultural AI System
# ==========================================================

!pip install gradio faiss-cpu sentence-transformers transformers torch torchvision torchaudio pillow --quiet

import gradio as gr
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from PIL import Image

# ==========================================================
# 1️⃣ Embedding Model
# ==========================================================
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ==========================================================
# 2️⃣ Expanded Knowledge Base (You can add more data)
# ==========================================================
documents = [
    "Yellow rust in wheat is a fungal disease causing yellow stripes. Controlled by Propiconazole spray.",
    "Rice blast causes diamond leaf spots. Use Tricyclazole fungicide.",
    "Aphids suck sap from crops and reduce yield. Control using neem oil or insecticide.",
    "Cotton bollworm damages cotton bolls. Use pheromone traps and biological pesticides.",
    "Urea fertilizer provides nitrogen for vegetative growth.",
    "DAP fertilizer supports strong root and early crop growth.",
    "PM-Kisan scheme provides 6000 rupees per year to eligible farmers.",
    "Over irrigation can cause root rot in many crops.",
    "Soil testing helps determine correct fertilizer dosage.",
    "Drip irrigation saves water and improves crop yield."
]

doc_embeddings = embedder.encode(documents)
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

# ==========================================================
# 3️⃣ More Powerful Open LLM
# ==========================================================
generator = pipeline("text-generation", model="google/flan-t5-large")

# ==========================================================
# 4️⃣ Speech & Image Models
# ==========================================================
speech_recognizer = pipeline("automatic-speech-recognition")
image_classifier = pipeline("image-classification")

# ==========================================================
# 🔍 Retrieval Function
# ==========================================================
def retrieve_context(query):
    query_embedding = embedder.encode([query])
    D, I = index.search(np.array(query_embedding), k=3)
    results = [documents[i] for i in I[0]]
    return "\n".join(results)

# ==========================================================
# 🌾 Main AI Function (Detailed + Practical)
# ==========================================================
def amrutha_system(text_input, audio_input, image_input):

    final_query = ""

    if audio_input is not None:
        speech_text = speech_recognizer(audio_input)["text"]
        final_query += speech_text + " "

    if text_input:
        final_query += text_input + " "

    if image_input is not None:
        img = Image.open(image_input)
        prediction = image_classifier(img)
        label = prediction[0]["label"]
        final_query += f"The image may contain {label}. "

    if final_query.strip() == "":
        return "Please provide your agricultural query."

    context = retrieve_context(final_query)

    prompt = f"""
    You are Amrutha, an expert agricultural assistant.

    Provide a detailed but clear answer suitable for farmers.
    Keep language simple.
    Give practical steps.
    Avoid unnecessary repetition.
    Crucially, if an image was provided and a potential crop/issue was identified, prioritize this information.
    Always start your answer by clearly stating the main crop or problem discussed, especially if an image was provided.

    Context:
    {context}

    Question:
    {final_query}

    Structure your answer strictly as:

    1. Problem Explanation
    2. Possible Causes
    3. Recommended Solution (step-by-step)
    4. Prevention Tips
    5. Additional Advice (if needed)
    """

    result = generator(
        prompt,
        max_length=512,   # Increased token length
        do_sample=True,
        temperature=0.4
    )[0]["generated_text"]

    return result

# ==========================================================
# 🎨 Enlarged Professional UI
# ==========================================================

with gr.Blocks(
    theme=gr.themes.Soft(),
    css="""
    #response_box textarea {
        min-height: 400px !important;
        font-size: 17px;
    }
    .gradio-container {
        max-width: 98% !important;
    }
    """
) as demo:

    gr.Markdown("## 🌾 Amrutha – Intelligent Agricultural Assistant")
    gr.Markdown("Ask any farming-related question using text, speech, or image.")

    text_input = gr.Textbox(
        label="Enter Your Agricultural Query (Text)",
        lines=4,
        placeholder="Example: My rice leaves have brown spots. What should I do?"
    )

    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Speak Your Query (Audio)")
        image_input = gr.Image(type="filepath", label="Upload Crop/Pest Image")

    submit_btn = gr.Button("Get Detailed Solution")

    output_box = gr.Textbox(
        label="🌾 Amrutha Response",
        elem_id="response_box",
        lines=20,
        max_lines=40,
        show_copy_button=True
    )

    submit_btn.click(
        amrutha_system,
        inputs=[text_input, audio_input, image_input],
        outputs=output_box
    )

demo.launch()

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/558 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLl

Loading weights:   0%|          | 0/212 [00:00<?, ?it/s]

Wav2Vec2ForCTC LOAD REPORT from: facebook/wav2vec2-base-960h
Key                        | Status  | 
---------------------------+---------+-
wav2vec2.masked_spec_embed | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
No model was supplied, defaulted to google/vit-base-patch16-224 and revision 3f49326.
Using a pipeline without specifying a model name and revision in production is not recommended.


Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
/tmp/ipython-input-211217444.py:123: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-211217444.py:123: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7359e09c9314bfe624.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
